In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext, simpledialog
import mne
import matplotlib.pyplot as plt
import matplotlib
"""
  * @author: dengyufeng
  * @Created on 2024/8/13 16:42
 """
# 设置Matplotlib的后端为'tkAgg'
matplotlib.use('tkAgg')

class EEGViewer:
    def __init__(self, root):
        self.root = root
        self.root.title("EEG数据查看器")
        self.root.geometry("500x450")

        # 创建按钮来加载EEG数据
        self.load_button = tk.Button(root, text="加载EEG数据", command=self.select_file_format)
        self.load_button.pack(pady=10)

        # 创建按钮来显示EEG数据
        self.view_button = tk.Button(root, text="查看EEG数据", command=self.view_eeg_data)
        self.view_button.pack(pady=10)
        self.view_button.config(state=tk.DISABLED)  # 初始禁用查看按钮

        # 创建一个滚动文本框来显示EEG数据的信息
        self.info_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=60, height=15)
        self.info_text.pack(pady=10)
        self.info_text.config(state=tk.DISABLED)  # 初始禁用文本框

        # 创建一个按钮来关闭窗口
        self.close_button = tk.Button(root, text="关闭窗口", command=self.close_window)
        self.close_button.pack(pady=10)

    def select_file_format(self):
        # 提示用户选择文件格式
        options = ["FIF", "EDF", "BDF"]
        file_format = simpledialog.askstring("选择文件格式", f"请选择文件格式（{', '.join(options)}）：")
        
        if file_format is None:
            return  # 用户取消操作

        file_format = file_format.upper()
        if file_format not in options:
            messagebox.showerror("错误", "不支持的文件格式！")
            return

        # 根据选择的文件格式设置文件扩展名
        if file_format == "FIF":
            filetypes = [("FIF文件", "*.fif")]
        elif file_format == "EDF":
            filetypes = [("EDF文件", "*.edf")]
        elif file_format == "BDF":
            filetypes = [("BDF文件", "*.bdf")]

        self.load_eeg_data(filetypes)

    def load_eeg_data(self, filetypes):
        # 弹出文件选择对话框以选择EEG文件
        self.file_path = filedialog.askopenfilename(
            title="选择EEG数据文件",
            filetypes=filetypes
        )
        if self.file_path:
            try:
                # 根据文件扩展名加载原始EEG数据
                if self.file_path.endswith('.fif'):
                    self.raw = mne.io.read_raw_fif(self.file_path, preload=True)
                elif self.file_path.endswith('.edf'):
                    self.raw = mne.io.read_raw_edf(self.file_path, preload=True)
                elif self.file_path.endswith('.bdf'):
                    self.raw = mne.io.read_raw_bdf(self.file_path, preload=True)
                else:
                    raise ValueError("不支持的文件格式")

                messagebox.showinfo("信息", "EEG数据加载成功！")

                # 在文本框中显示EEG数据的信息
                self.info_text.config(state=tk.NORMAL)  # 启用文本框
                self.info_text.delete(1.0, tk.END)  # 清空文本框
                self.info_text.insert(tk.END, str(self.raw.info))  # 显示info信息
                self.info_text.config(state=tk.DISABLED)  # 显示完成后禁用文本框

                self.view_button.config(state=tk.NORMAL)  # 启用查看按钮
            except Exception as e:
                messagebox.showerror("错误", f"加载EEG数据失败：{str(e)}")

    def view_eeg_data(self):
        if hasattr(self, 'raw'):
            # 绘制EEG数据
            self.raw.plot()
        else:
            messagebox.showwarning("警告", "请先加载EEG数据！")

    def close_window(self):
        # 关闭主窗口
        self.root.quit()
        self.root.destroy()

# 创建主窗口
root = tk.Tk()
app = EEGViewer(root)

# 运行GUI主循环
root.mainloop()
